In [6]:
from langchain_groq import ChatGroq
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

llm = ChatGroq(model="llama3-70b-8192",api_key="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")

memory = ConversationBufferMemory()
conversation_chain = ConversationChain(memory=memory,llm=llm)

response = conversation_chain.run(input="can u summarize a pdf if i gave u my pdf  ")
response2 = conversation_chain.run(input="can u write a code for me ")
print(response)
print(response2)

I'd be happy to help summarize a document for you. Before we get started, can you please tell me a bit more about the PDF? For example, how many pages does it have, and what's the general topic or subject matter? Additionally, are you looking for a brief summary of the entire document, or would you like me to focus on specific sections or key takeaways?

Also, I'll let you know that I'm trained on a vast amount of text data, but I don't have the capability to directly upload or access files. So, if you could copy and paste the text from the PDF or provide a link to a publicly accessible version, I'll do my best to assist you with the summary.
I'd be happy to help write some code for you. Before we get started, can you please tell me a bit more about what you need? For example, what programming language are you interested in using (e.g., Python, Java, JavaScript, etc.)? Additionally, what kind of code are you trying to write (e.g., a simple script, a web application, a machine learning 